# Weights & Biases test notebook
This example notebook shows the code I used to create the example graphs shown on the report for Weights & Biases. \
Please remember that to run this and create your own graphs, you need an account on the W&B web page, at https://wandb.ai/site. \
Remember also that to view the visualisations and access the model registry, you need to run "wandb server" in a terminal.

In [ ]:
# The requirements:

%pip install wandb tensorflow

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

import wandb

In [ ]:
# The MNIST classifier dataset is a set of images, made up of 28x28 pixels, of value between 0 and 255.
# First of all we need to split the 70'000 images into a training set and a testing one:
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

# For simpler use of the data, we want each value to be between 0 and 1.
X_train_divided = X_train / 255
X_test_divided = X_test / 255

# This next block is a simple way to create models with varying accuracy and loss.
first_layer_nodes = [100, 50, 100, 50, 100, 50]
first_layer_activation = ["relu", "sigmoid", "softmax", "relu", "sigmoid", "softmax"]
second_layer_nodes = [10, 15, 25, 25, 15, 10]
second_layer_activation = ["sigmoid", "softmax", "relu", "relu", "sigmoid", "softmax"]

Before logging in to wandb, it's necessary to create an account on the Weghts & Biases website, at https://wandb.ai/site. I personally used my Google account and it worked fine for SaaS testing, it's slightly simpler, whereas for the local server a licence may be required. \
If it's the first time logging into your account, you should log in on the website, navigate to https://wandb.ai/authorize, run "wandb.login()", copy your API key and paste it under the login when prompted. \
Alternatively, you can log in directly with "wandb.login(key='your API key')", with the key (https://wandb.ai/authorize) as a string in quotes.

If you get errors at the login stage check that the wandb server is still running in the terminal, or if you didn't start it yet, run "wandb server" in a terminal.

In [ ]:
wandb.login()

In the next code cell we will log the metrics for the 6 different model compositions. We will be logging metadata such as the hyperparameters for each run, which we will name "run_1" through "run_6", and the metrics accuracy and loss. Some data is automatically logged, such as GPU usage throughout the run (in my case no GPUs were used, sorry to disappoint, but CPU usage is logged instead).

In [ ]:
for run_number in range(6):
    # Each run logged with wandb should be initialised and then ended to close the connection with the server.
    wandb.init(project = "MNIST model variations",
               config = { # these are all hyperparameters
                   "architecture": "CNN",
                   "dataset":  "keras MNIST dataset",
                   "first layer nodes": first_layer_nodes[run_number],
                   "first layer activation": first_layer_activation[run_number],
                   "second layer nodes": second_layer_nodes[run_number],
                   "second layer activation": second_layer_activation[run_number],
                   "optimizer": "adam",
                   "loss calculation": "sparse_categorical_crossentropy",
                   "epochs": 5,
               },
               name = "run_{}".format(run_number + 1))
    
    # Create, compile and train the model:
    model = Sequential([
        keras.layers.Flatten(input_shape = (28,28)),
        keras.layers.Dense(first_layer_nodes[run_number], first_layer_activation[run_number]),
        keras.layers.Dense(second_layer_nodes[run_number], second_layer_activation[run_number])
    ])
    model.compile(
        optimizer = 'adam',
        loss = 'sparse_categorical_crossentropy',
        metrics = ['accuracy']
    )
    history = model.fit(X_train_divided, y_train, epochs = 5, verbose = 1)

    # For each run we want to log the accuracy and loss after each epoch.
    for i in range(5):
         wandb.log({"acc": history.history['accuracy'][i], "loss": history.history['loss'][i]})
    
    wandb.finish()

In the case of Weights & Biases, unique features include automatic hardware usage logging, such as the CPU usage, or logging of usage of each CPU core.